---
title: Naive Langchain Rating
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.17.1
  kernelspec:
    display_name: nix
    language: python
    name: nix
---


Set the `cd` to the project root:

In [1]:
import os, subprocess

os.chdir(subprocess
         .check_output(["git", "rev-parse", "--show-toplevel"])
         .decode('utf-8')
         .strip())
os.getcwd()

'/home/kerry/mnt/counter/madrs-ai'

Defining the graph:

Invocation. The server here was started on the backend with something like
```sh
python -m llama_cpp.server \
    --model ./models/mistralai_Mistral-Small-3.1-24B-Instruct-2503-Q4_K_M.gguf \
    --model_alias mistral \
    --host 0.0.0.0 --port 8080 \
    --n_gpu_layers -1 \
    --n_ctx 50000 \
    --chat_format chatml-function-calling
```

In [2]:
from typing import Annotated, Optional
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langgraph.graph import END, START, MessagesState, StateGraph
from pydantic import BaseModel, Field

def build_rater(llm: BaseChatModel):

    MadrsFieldScore = Annotated[
        int,
        Field(
            description="The numerical score for this field.",
            ge=0,
            le=6,
        )
    ]
    class MadrsFieldRating(BaseModel):
        score: int = Field(
            description="The numerical score for this field.",
            ge=0,
            le=6,
        )
        justification: str = Field(
            description="A brief justification for the score given for this field."
        )

    class MadrsRating(BaseModel):
        apparent_sadness: MadrsFieldRating = Field(
            description = "The MADRS rating for apparent sadness."
        )
        reported_sadness: MadrsFieldRating = Field(
            description = "The MADRS rating for reported sadness."
        )
        inner_tension: MadrsFieldRating = Field(
            description = "The MADRS rating for inner tension."
        )
        reduced_sleep: MadrsFieldRating = Field(
            description = "The MADRSrating for reduced sleep."
        )
        reduced_appetite: MadrsFieldRating = Field(
            description = "The MADRS rating for reduced appetite."
        )
        concentration_difficulties: MadrsFieldRating = Field(
            description = "The MADRS rating for concentration difficulties."
        )
        lassitude: MadrsFieldRating = Field(
            description = "The MADRS rating for lassitude."
        )
        inability_to_feel: MadrsFieldRating = Field(
            description = "The MADRS rating for inability to feel."
        )
        pessimistic_thoughts: MadrsFieldRating = Field(
            description = "The MADRS rating for pessimistic thoughts."
        )
        suicidal_thoughts: MadrsFieldRating = Field(
            description = "The MADRS rating for suicidal thoughts."
        )

    class RaterState(MessagesState):
        transcript: str
        primer: str
        rating: Optional[MadrsRating]


    def build_prompt(state: RaterState):
        prompt_template = PromptTemplate.from_template(
            """
            You are an expert in psychiatric assessments and depression. For your
            recollection, consider the following primer on MADRS interviews:

            ---

            {primer}

            ---

            Using this information, provide the most accurate assessment of the
            following transcription of a MADRS interview. Provide your scores for
            each of the items described in the primer, providing a best guess for
            apparent sadness, together with a brief justification/discussion.

            ---

            {transcript}
            """
        )
        return {
            "messages": prompt_template.invoke(
                {
                    "transcript": state["transcript"],
                    "primer": state["primer"]
                }
            ).to_messages()
        }

    def invoke_llm(state: RaterState):
        return { "messages": llm.invoke(state["messages"]) }

    def invoke_structured_llm(state: RaterState):
        parse_message = HumanMessage("Please format your previous response in the prescibed way.")
        return {
            # Required to work with llama-cpp server, json_schema not supported yet
            "rating": llm.with_structured_output(MadrsRating, method="function_calling").invoke(
                state["messages"] + [parse_message]
            ),
        }

    return (
        StateGraph(RaterState)
        .add_node("build_prompt", build_prompt)
        .add_node("invoke_llm", invoke_llm)
        .add_node("invoke_structured_llm", invoke_structured_llm)
        .add_edge(START, "build_prompt")
        .add_edge("build_prompt", "invoke_llm")
        .add_edge("invoke_llm", "invoke_structured_llm")
        .add_edge("invoke_structured_llm", END)
        .compile()
    )

In [3]:
from langchain_openai import ChatOpenAI
from pathlib import Path

llm = ChatOpenAI(
    model = "mistral",
    base_url = "http://localhost:8080/v1",
    api_key = "NULL",
)
transcripts_path = Path("data/interviews/transcripts/corrected")
primer_path = Path("data/madrs-instructions/primer.md")
transcripts_paths = [transcripts_path / file for file in os.listdir("data/interviews/transcripts/corrected") if file.endswith("txt")]
graph = build_rater(llm)
with open(primer_path, "r") as file:
    primer = file.read()
results = {}
for transcript_path in transcripts_paths:
    with open(transcript_path) as file:
        transcript = file.read()
    response = graph.invoke({
        "transcript": transcript,
        "primer" : primer
    })
    results[transcript_path.name.rstrip(".txt")] = {
        "rating": response["rating"].model_dump(),
        "unstructured_rating": response["messages"][-1].content
    }


Push the output to a temp file.

In [4]:
import json

output_path = Path("temp/output.json")
with output_path.open(mode='w') as file:
    file.write(json.dumps(results, indent=3))